## 0. BEM blade dividing

In [ ]:
import numpy as np
import math

#==================.exe Compiler#=======================

# Set the range and distribution of r
num_elements = 10  # Number of blade elements
R = 8.5
r_start = 2.473  # Starting radius (m)
r_end = 8.5      # Ending radius (m)
c_start = 1.545  # Starting chord length (m)
c_end = 0.927    # Ending chord length (m)a

# Use linspace to generate evenly spaced r values, num_elements + 1 points, forming num_elements segments
delta_r = (r_end - r_start) / num_elements  # Width of each segment δr
radii = np.linspace(r_start, r_end, num_elements + 1)
chord_lengths = np.linspace(c_start, c_end, num_elements + 1)

theta_root = math.radians(10)
theta_tip = math.radians(0)

def get_twist_angle(r, r_start, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * ((r - r_start) / (R - r_start))

print("r (m)    twist (deg)")
for r in radii:
    twist = get_twist_angle(r, r_start, R, theta_root, theta_tip)
    print(f"{r:6.3f}   {math.degrees(twist):6.2f}")
print("Width of each segment δr:", delta_r)
print("Radius values (radii):", radii)
print("Chord length values (chord):", chord_lengths)


r (m)    twist (deg)
 2.473    10.00
 3.076     9.00
 3.678     8.00
 4.281     7.00
 4.884     6.00
 5.486     5.00
 6.089     4.00
 6.692     3.00
 7.295     2.00
 7.897     1.00
 8.500     0.00
Width of each segment δr: 0.6027
Radius values (radii): [2.473  3.0757 3.6784 4.2811 4.8838 5.4865 6.0892 6.6919 7.2946 7.8973
 8.5   ]
Chord length values (chord): [1.545  1.4832 1.4214 1.3596 1.2978 1.236  1.1742 1.1124 1.0506 0.9888
 0.927 ]


## 1. Define U_infinity to get a and a'

In [ ]:
import numpy as np
import math

# Parameter settings
U_infinity = 2  # Free-stream velocity (m/s)
omega = 1.26    # Rotational speed (rad/s)
R = 8.5         # Total blade radius (m)
B = 3           # Number of blades
tolerance = 1e-6  # Convergence tolerance for iteration
small_value = 1e-6  # Small value to prevent division by zero

# Initial CL, CD values
CL_initial = np.array([0.00000, 0.11060, 0.22060, 0.32930, 0.43620,
                       0.54070, 0.64220, 0.75230, 0.87660, 0.99990, 1.07670])
CD_initial = np.array([0.00096, 0.00097, 0.00100, 0.00106, 0.00115,
                       0.00127, 0.00141, 0.00163, 0.00192, 0.00233, 0.00300])

# Twist angle parameters
theta_root = math.radians(10)  # Twist angle at root (10°)
theta_tip = math.radians(0)    # Twist angle at tip (0°)

# Function to calculate twist angle at a given radius
def get_twist_angle(r, r_start, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * ((r - r_start) / (R - r_start))

# Prandtl tip loss factor function
def prandtl_tip_loss(B, r, R, phi):
    if abs(r - R) < small_value:
        return 0.6  # When r = R, set F = 0.6 to avoid division by zero
    f = (B / 2) * (R - r) / (r * math.sin(phi))
    F = (2 / math.pi) * math.acos(math.exp(-f))
    return F

# Function to compute Cn and Ct from CL, CD, and phi
def calculate_forces_projection(CL, CD, phi):
    # Build transformation matrix T and apply it to [CL, CD]
    T = np.array([[math.cos(phi), math.sin(phi)],
                  [math.sin(phi), -math.cos(phi)]])
    forces = np.array([CL, CD])
    projected_forces = T @ forces  # Matrix multiplication
    Cn, Ct = projected_forces
    return Cn, Ct

# Function to calculate induction factors with twist and updated formulas
def calculate_induction_with_twist(a_init, a_prime_init, r, chord, CL, CD, consider_tip_loss):
    a = a_init
    a_prime = a_prime_init

    for _ in range(1000):
        # Compute twist angle and correct inflow angle
        twist_angle = get_twist_angle(r, r_start, R, theta_root, theta_tip)
        phi = math.atan((1 - a) * U_infinity / ((1 + a_prime) * omega * r + small_value)) - twist_angle

        # Compute Cn and Ct from corrected phi
        Cn, Ct = calculate_forces_projection(CL, CD, phi)

        # Compute Prandtl tip loss factor
        F = prandtl_tip_loss(B, r, R, phi) if consider_tip_loss else 1

        # Compute blade solidity sigma
        sigma = chord / (2 * math.pi * r)

        # Update a and a_prime
        a_new = 1 / (1 + F * (4 * math.sin(phi)**2) / (sigma * Cn + small_value))
        a_prime_new = 1 / (F * (4 * math.sin(phi) * math.cos(phi) / (sigma * Ct + small_value)) - 1)

        # Check for convergence
        if abs(a_new - a) < tolerance and abs(a_prime_new - a_prime) < tolerance:
            a, a_prime = a_new, a_prime_new
            break

        a, a_prime = a_new, a_prime_new

    return a, a_prime, F, Cn, Ct

# Initialize result arrays
a_no_tip = []
a_prime_no_tip = []
a_with_tip = []
a_prime_with_tip = []
F_values = []
Cn_initial = []
Ct_initial = []

# Compute induction factors and tip loss for each radius value
for i, r in enumerate(radii):
    CL = CL_initial[i]
    CD = CD_initial[i]
    chord = chord_lengths[i]

    # Induction factors without tip loss
    a_nt, a_prime_nt, _, Cn, Ct = calculate_induction_with_twist(0.3, 0.01, r, chord, CL, CD, consider_tip_loss=False)
    a_no_tip.append(a_nt)
    a_prime_no_tip.append(a_prime_nt)

    # Induction factors with tip loss
    a_wt, a_prime_wt, F, Cn, Ct = calculate_induction_with_twist(0.3, 0.01, r, chord, CL, CD, consider_tip_loss=True)
    a_with_tip.append(a_wt)
    a_prime_with_tip.append(a_prime_wt)
    F_values.append(F)
    Cn_initial.append(Cn)
    Ct_initial.append(Ct)

# Output results
print("Radius (r) | a (no tip loss) | a' (no tip loss) | a (with tip loss) | a' (with tip loss) | Tip Loss F |    Cn    |    Ct    |    CL   |   CD")
for i, r in enumerate(radii):
    print(f"{r:.3f}     | {a_no_tip[i]:.6f}      | {a_prime_no_tip[i]:.6f}       | "
          f"{a_with_tip[i]:.6f}       | {a_prime_with_tip[i]:.6f}      | {F_values[i]:.4f}    | "
          f"{Cn_initial[i]:.6f} | {Ct_initial[i]:.6f} | {CL_initial[i]:.4f}  | {CD_initial[i]:.4f}")

Radius (r) | a (no tip loss) | a' (no tip loss) | a (with tip loss) | a' (with tip loss) | Tip Loss F |    Cn    |    Ct    |    CL   |   CD
2.473     | 0.000064      | -0.000061       | 0.000064       | -0.000061      | 1.0000    | 0.000370 | -0.000886 | 0.0000  | 0.0010
3.076     | 0.021353      | 0.002173       | 0.021355       | 0.002173      | 0.9999    | 0.105633 | 0.032787 | 0.1106  | 0.0010
3.678     | 0.052017      | 0.003448       | 0.052030       | 0.003449      | 0.9998    | 0.214125 | 0.053066 | 0.2206  | 0.0010
4.281     | 0.092839      | 0.004198       | 0.092885       | 0.004200      | 0.9996    | 0.322881 | 0.064709 | 0.3293  | 0.0011
4.884     | 0.143452      | 0.004625       | 0.143622       | 0.004629      | 0.9992    | 0.430426 | 0.070751 | 0.4362  | 0.0011
5.486     | 0.199482      | 0.004837       | 0.200215       | 0.004845      | 0.9983    | 0.535657 | 0.073689 | 0.5407  | 0.0013
6.089     | 0.239284      | 0.004905       | 0.242977       | 0.004930      | 0.99

In [ ]:
# @title Tip Loss Check
import math
import numpy as np

# Fixed parameters
U_infinity = 2        # Free-stream velocity (m/s)
omega = 1.26          # Rotational speed (rad/s)
R = 8.5               # Blade tip radius (m)
B = 3                 # Number of blades
tolerance = 1e-6      # Iteration convergence tolerance
small_value = 1e-6    # To prevent division by zero

theta_root = math.radians(10)  # Twist angle at root (10°)
theta_tip = math.radians(0)    # Twist angle at tip (0°)

# For data at r = R
r = R
chord = 0.927
CL = 1.07670
CD = 0.00300

def get_twist_angle(r, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * (r / R)

def prandtl_tip_loss(B, r, R, phi):
    # When r is close to R, directly return assumed F=0.6
    if abs(r - R) < small_value:
        return 0.6
    f = (B / 2) * (R - r) / (r * math.sin(phi))
    F = (2 / math.pi) * math.acos(math.exp(-f))
    return F

def calculate_forces_projection(CL, CD, phi):
    # Compute T matrix and apply it to [CL, CD]
    T = np.array([[math.cos(phi), math.sin(phi)],
                  [math.sin(phi), -math.cos(phi)]])
    forces = np.array([CL, CD])
    projected_forces = T @ forces  # Matrix multiplication
    Cn, Ct = projected_forces
    return Cn, Ct

def calculate_induction_with_twist_debug(a_init, a_prime_init, r, chord, CL, CD, consider_tip_loss):
    a = a_init
    a_prime = a_prime_init
    print(f"[Debug Info] r = {r:.3f} m, chord = {chord:.3f} m, CL = {CL}, CD = {CD}")
    for it in range(1000):
        twist_angle = get_twist_angle(r, R, theta_root, theta_tip)
        # Calculate relative inflow angle phi (minus twist angle)
        phi = math.atan((1 - a) * U_infinity / ((1 + a_prime) * omega * r + small_value)) - twist_angle

        # When considering tip loss and r==R, directly return F = 0.6
        if consider_tip_loss:
            F = prandtl_tip_loss(B, r, R, phi)
        else:
            F = 1

        Cn, Ct = calculate_forces_projection(CL, CD, phi)
        sigma = chord / (2 * math.pi * r)

        # Update formulas for a and a′
        a_new = 1 / (1 + F * (4 * math.sin(phi)**2) / (sigma * Cn + small_value))
        a_prime_new = 1 / (F * (4 * math.sin(phi) * math.cos(phi) / (sigma * Ct + small_value)) - 1)

        print(f"Iteration {it:2d}: φ = {phi:.6f} rad, a = {a_new:.6f}, a' = {a_prime_new:.6f}, F = {F:.4f}")

        # Check for convergence
        if abs(a_new - a) < tolerance and abs(a_prime_new - a_prime) < tolerance:
            a, a_prime = a_new, a_prime_new
            break
        a, a_prime = a_new, a_prime_new

    return a, a_prime, F, Cn, Ct

print("----- Iteration Debug: With Tip Loss (F = 0.6 when r = R) -----")
a_with_tip, a_prime_with_tip, F_with_tip, Cn_with_tip, Ct_with_tip = calculate_induction_with_twist_debug(
    0.3, 0.01, r, chord, CL, CD, consider_tip_loss=True
)
print("\nFinal Result (With Tip Loss): a = {:.6f}, a' = {:.6f}, F = {:.4f}".format(
    a_with_tip, a_prime_with_tip, F_with_tip))



半径 (r)  | (1-a)U∞  | (1+a')ωr |   U_rel  | tan(phi) |  phi(deg) | twist(deg)| phi - twist(r)
2.473     | 1.999873 | 3.115789 | 3.702382 | 0.641851 | 32.694413 | 10.000000 | 22.694413
3.076     | 1.957290 | 3.883803 | 4.349127 | 0.503962 | 26.746374 | 9.000000 | 17.746374
3.678     | 1.895941 | 4.650770 | 5.022375 | 0.407662 | 22.178834 | 8.000000 | 14.178834
4.281     | 1.814230 | 5.416841 | 5.712583 | 0.334924 | 18.516928 | 7.000000 | 11.516928
4.884     | 1.712756 | 6.182070 | 6.414945 | 0.277052 | 15.485503 | 6.000000 | 9.485503
5.486     | 1.599569 | 6.946483 | 7.128271 | 0.230270 | 12.967475 | 5.000000 | 7.967475
6.089     | 1.514046 | 7.710218 | 7.857467 | 0.196369 | 11.109742 | 4.000000 | 7.109742
6.692     | 1.487156 | 8.474493 | 8.603991 | 0.175486 | 9.953270 | 3.000000 | 6.953270
7.295     | 1.463670 | 9.241341 | 9.356533 | 0.158383 | 8.999909 | 2.000000 | 6.999909
7.897     | 1.343641 | 10.015598 | 10.105325 | 0.134155 | 7.640882 | 1.000000 | 6.640882
8.500     | 1.293460 |

## 2. Calculate Relative velocity **Urel** and Flow angle **phi**

In [ ]:
# Initialize the lists to store intermediate results
one_minus_a_U_infinity = []
one_plus_a_prime_omega_r = []
Urel_values = []
tan_phi_values = []
phi_values = []
phi_minus_twist_values = []
twist_angles_deg = []  # New: list to store twist angles in degrees

# Calculate intermediate variables for each radial position
for i, r in enumerate(radii):
    a = a_no_tip[i]
    a_prime = a_prime_no_tip[i]
    chord = chord_lengths[i]
    CL = CL_initial[i]
    CD = CD_initial[i]

    one_minus_a_Uinf = (1 - a) * U_infinity
    one_plus_a_prime_omega_r_val = (1 + a_prime) * omega * r

    Urel = math.sqrt(one_minus_a_Uinf**2 + one_plus_a_prime_omega_r_val**2)

    tan_phi = one_minus_a_Uinf / (one_plus_a_prime_omega_r_val + small_value)
    phi = math.degrees(math.atan(tan_phi))

    twist_angle = get_twist_angle(r, r_start, R, theta_root, theta_tip)
    twist_angle_deg = math.degrees(twist_angle)
    phi_minus_twist = phi - twist_angle_deg

    one_minus_a_U_infinity.append(one_minus_a_Uinf)
    one_plus_a_prime_omega_r.append(one_plus_a_prime_omega_r_val)
    Urel_values.append(Urel)
    tan_phi_values.append(tan_phi)
    phi_values.append(phi)
    twist_angles_deg.append(twist_angle_deg)
    phi_minus_twist_values.append(phi_minus_twist)

print("\nRadius (r)  | (1-a)U∞  | (1+a')ωr |   U_rel  | tan(phi) |  phi(deg) | twist(deg)| phi - twist(r)")
for i, r in enumerate(radii):
    print(f"{r:.3f}     | {one_minus_a_U_infinity[i]:.6f} | {one_plus_a_prime_omega_r[i]:.6f} | "
          f"{Urel_values[i]:.6f} | {tan_phi_values[i]:.6f} | {phi_values[i]:.6f} | {twist_angles_deg[i]:8.6f} | {phi_minus_twist_values[i]:.6f}")



Radius (r)  | (1-a)U∞  | (1+a')ωr |   U_rel  | tan(phi) |  phi(deg) | twist(deg)| phi - twist(r)
2.473     | 1.999873 | 3.115789 | 3.702382 | 0.641851 | 32.694413 | 10.000000 | 22.694413
3.076     | 1.957295 | 3.883802 | 4.349129 | 0.503963 | 26.746436 | 9.000000 | 17.746436
3.678     | 1.895965 | 4.650767 | 5.022381 | 0.407667 | 22.179105 | 8.000000 | 14.179105
4.281     | 1.814322 | 5.416833 | 5.712604 | 0.334941 | 18.517830 | 7.000000 | 11.517830
4.884     | 1.713096 | 6.182049 | 6.415016 | 0.277108 | 15.488484 | 6.000000 | 9.488484
5.486     | 1.601036 | 6.946429 | 7.128547 | 0.230483 | 12.979059 | 5.000000 | 7.979059
6.089     | 1.521433 | 7.710027 | 7.858707 | 0.197332 | 11.162854 | 4.000000 | 7.162854
6.692     | 1.515592 | 8.473569 | 8.608042 | 0.178861 | 10.140756 | 3.000000 | 7.140756
7.295     | 1.549653 | 9.237243 | 9.366327 | 0.167761 | 9.523342 | 2.000000 | 7.523342
7.897     | 1.602048 | 10.000108 | 10.127622 | 0.160203 | 9.101619 | 1.000000 | 8.101619
8.500     | 1.672

## 3. Lift & Drag and Normal & Tangential Force

In [ ]:
# ====================== Force Calculation Section ======================
rho = 1025  # Water density (kg/m³)
CL_star = np.array([0.6729, 1.2938, 1.8137, 1.7503, 1.6211, 1.4972, 1.4071, 1.3479, 1.3165, 1.3082, 1.3255])
CD_star = np.array([0.30404, 0.17387, 0.03682, 0.01989, 0.01556, 0.01328, 0.01191, 0.01119, 0.01078, 0.01065, 0.01085])

# Initialize storage lists
lift_initial, drag_initial = [], []
Fn_initial, Ft_initial = [], []
lift_star, drag_star = [], []
Fn_star, Ft_star = [], []
Cn_star_values = []
Ct_star_values = []  # Directly store * versions of Cn and Ct

# Compute forces at each radial position
for i, r in enumerate(radii):
    Urel = Urel_values[i]  # Ensure Urel_values are computed from BEM
    chord = chord_lengths[i]

    # Calculate Cn* and Ct*
    phi = math.atan((1 - a_with_tip[i]) * U_infinity / ((1 + a_prime_with_tip[i]) * omega * r))
    Cn_star, Ct_star = calculate_forces_projection(CL_star[i], CD_star[i], phi)
    Cn_star_values.append(Cn_star)
    Ct_star_values.append(Ct_star)

    # **Initial force calculation (using Cn_initial and Ct_initial)**
    L_initial = 0.5 * rho * Urel**2 * chord * delta_r * CL_initial[i] / 1000
    D_initial = 0.5 * rho * Urel**2 * chord * delta_r * CD_initial[i] / 1000
    Fn_initial_val = Cn_initial[i] * 0.5 * rho * Urel**2 * chord * delta_r / 1000
    Ft_initial_val = Ct_initial[i] * 0.5 * rho * Urel**2 * chord * delta_r / 1000

    lift_initial.append(L_initial)
    drag_initial.append(D_initial)
    Fn_initial.append(Fn_initial_val)
    Ft_initial.append(Ft_initial_val)

    # **Refined force calculation (using CL*/CD*)**
    L_star = 0.5 * rho * Urel**2 * chord * delta_r * CL_star[i] / 1000
    D_star = 0.5 * rho * Urel**2 * chord * delta_r * CD_star[i] / 1000

    Fn_star_val = Cn_star * 0.5 * rho * Urel**2 * chord * delta_r / 1000
    Ft_star_val = Ct_star * 0.5 * rho * Urel**2 * chord * delta_r / 1000

    lift_star.append(L_star)
    drag_star.append(D_star)
    Fn_star.append(Fn_star_val)
    Ft_star.append(Ft_star_val)

# ====================== Output Results ======================
print("Radius (r) |     L (kN)     |     D (kN)    |    Fn (kN)   |     Ft (kN)   |    Cn    |    Ct    |    CL    |  CD  |")
for i, r in enumerate(radii):
    print(f"{r:.3f}  | {lift_initial[i]:.6f} kN   | {drag_initial[i]:.6f} kN   | "
          f"{Fn_initial[i]:.6f} kN   | {Ft_initial[i]:.6f} kN   | "
          f"{Cn_initial[i]:.6f} | {Ct_initial[i]:.6f} | {CL_initial[i]:.4f}  | {CD_initial[i]:.4f}")

print("Radius (r) |     L* (kN)    |     D* (kN)    |    Fn* (kN)   |    Ft* (kN)   |    Cn*   |    Ct*   |    CL*  | CD*  |")
for i, r in enumerate(radii):
    print(f"{r:.3f}  | {lift_star[i]:.6f} kN   | {drag_star[i]:.6f} kN   | "
          f"{Fn_star[i]:.6f} kN   | {Ft_star[i]:.6f} kN   | "
          f"{Cn_star_values[i]:.6f} | {Ct_star_values[i]:.6f} | {CL_star[i]:.4f}  | {CD_star[i]:.4f}")


Radius (r) |     L (kN)     |     D (kN)    |    Fn (kN)   |     Ft (kN)   |    Cn    |    Ct    |    CL    |  CD  |
2.473  | 0.000000 kN   | 0.006280 kN   | 0.002423 kN   | -0.005794 kN   | 0.000370 | -0.000886 | 0.0000  | 0.0010
3.076  | 0.958417 kN   | 0.008406 kN   | 0.915373 kN   | 0.284123 kN   | 0.105633 | 0.032787 | 0.1106  | 0.0010
3.678  | 2.443072 kN   | 0.011075 kN   | 2.371359 kN   | 0.587691 kN   | 0.214125 | 0.053066 | 0.2206  | 0.0010
4.281  | 4.513008 kN   | 0.014527 kN   | 4.425043 kN   | 0.886821 kN   | 0.322881 | 0.064709 | 0.3293  | 0.0011
4.884  | 7.195881 kN   | 0.018971 kN   | 7.100621 kN   | 1.167154 kN   | 0.430426 | 0.070751 | 0.4362  | 0.0011
5.486  | 10.489916 kN   | 0.024639 kN   | 10.392071 kN   | 1.429614 kN   | 0.535657 | 0.073689 | 0.5407  | 0.0013
6.089  | 14.384993 kN   | 0.031583 kN   | 14.278295 kN   | 1.749090 kN   | 0.637437 | 0.078086 | 0.6422  | 0.0014
6.692  | 19.153840 kN   | 0.041500 kN   | 19.017992 kN   | 2.277561 kN   | 0.746964 | 0.08945

In [ ]:
# Compute the average of adjacent rows to obtain 10 rows of results
avg_radii = []
avg_lift_star = []
avg_drag_star = []
avg_Fn_star = []
avg_Ft_star = []
avg_Cn_star = []
avg_Ct_star = []

for i in range(len(radii) - 1):  # Prevent index out of range
    avg_radii.append((radii[i] + radii[i + 1]) / 2)
    avg_lift_star.append((lift_star[i] + lift_star[i + 1]) / 2)
    avg_drag_star.append((drag_star[i] + drag_star[i + 1]) / 2)
    avg_Fn_star.append((Fn_star[i] + Fn_star[i + 1]) / 2)
    avg_Ft_star.append((Ft_star[i] + Ft_star[i + 1]) / 2)
    avg_Cn_star.append((Cn_star_values[i] + Cn_star_values[i + 1]) / 2)
    avg_Ct_star.append((Ct_star_values[i] + Ct_star_values[i + 1]) / 2)

# Output the 10-row averaged results
print("\nAveraged 10 rows of results:")
print("Radius (r) | Avg Lift δL* (N) | Avg Drag δD* (N) | Avg Normal Force δFn* (N) | Avg Tangential Force δFt* (N)")
for i in range(len(avg_radii)):  # Prevent index out of range
    print(f"{avg_radii[i]:.3f}     | {avg_lift_star[i]:.6f} N     | {avg_drag_star[i]:.6f} N     | "
          f"{avg_Fn_star[i]:.6f} N      | {avg_Ft_star[i]:.6f} N")



Averaged 10 rows of results:
Radius (r) | Avg Lift δL* (N) | Avg Drag δD* (N) | Avg Normal Force δFn* (N) | Avg Tangential Force δFt* (N)
2.774     | 7.806717 N     | 1.747804 N     | 7.734434 N      | 2.202048 N
3.377     | 15.648849 N     | 0.957230 N     | 14.721989 N      | 5.452541 N
3.980     | 22.036866 N     | 0.340179 N     | 20.793111 N      | 7.282257 N
4.582     | 25.365241 N     | 0.264639 N     | 24.336463 N      | 7.126228 N
5.185     | 27.894743 N     | 0.257165 N     | 27.102128 N      | 6.579881 N
5.788     | 30.282510 N     | 0.262209 N     | 29.671412 N      | 6.039198 N
6.391     | 32.918225 N     | 0.275840 N     | 32.414961 N      | 5.731276 N
6.993     | 35.898690 N     | 0.295898 N     | 35.458344 N      | 5.605486 N
7.596     | 39.230646 N     | 0.320264 N     | 38.864177 N      | 5.339163 N
8.199     | 42.992719 N     | 0.351006 N     | 42.695000 N      | 5.053915 N


## 4. Output csv file


In [ ]:
import pandas as pd
from google.colab import files

output_file = 'Fast_BEM_Force_Solver_results.csv'

Fast_BEM_results = pd.DataFrame({
    "Element number": list(range(0, num_elements + 1)),
    "radii": radii,
    "a": a_no_tip,
    "a'": a_prime_no_tip,
    "a_corr": a_with_tip,
    "a'_corr": a_prime_with_tip,
    "F_tiploss": F_values,
    "(1-a)U_infnty": one_minus_a_U_infinity,
    "(1+a')wr": one_plus_a_prime_omega_r,
    "Urel": Urel_values,
    "tan(phi)": tan_phi_values,
    "phi": phi_values,
    "phi-twist": phi_minus_twist_values,
    "L*": lift_star,
    "D*": drag_star,
    "L": lift_initial,
    "D": drag_initial,
    "Fn*": Fn_star,
    "Ft*": Ft_star,
    "Fn": Fn_initial,
    "Ft": Ft_initial,
    "Cn*": Cn_star_values,
    "Ct*": Ct_star_values,
    "Cn": Cn_initial,
    "Ct": Ct_initial,
    "CL*": CL_star,
    "CD*": CD_star,
    "CL_ini": CL_initial,
    "CD_ini": CD_initial,
})

Fast_BEM_results.to_csv(output_file, index=False)
try:
    from google.colab import files
    files.download(output_file)
except:
    print(f"File saved: {output_file}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>